In [ ]:
# 그냥 여기 한 .py에다가 다 몰아넣기
# 어차피 이 기능만 씀 파이썬에서
# 아닌가? 아님말고

# 이걸 Spring -> Python -> Spring으로? 
# 아니면 Spring -> Python -> MySQL <- Spring으로? 굳이 고객 정보 없는 곳에서 할 필요 X 
##### 

In [1]:
import cv2 as cv2
import numpy as np
from PIL import Image, ImageDraw

from sklearn.cluster import KMeans
# import matplotlib.pyplot as plt

import json
import requests

from flask import Flask, request, jsonify
import base64

import logging
import sys

from PIL import Image

In [ ]:
app = Flask(__name__)

@app.route('/test/personalcolor', methods=['POST'])
def hello():
    binary_decode = base64.b64decode(request.get_json()["binary"]) # image binary 

    encoded_img = np.fromstring(binary_decode, dtype = np.uint8)
    img_origin = cv2.imdecode(encoded_img, cv2.IMREAD_COLOR) # image
    
    res = getfacialPointsJsonInKakaoAPI(binary_decode)
    print('res : ', res)
    img = getFaceImg(img_origin, res)
    
    avg_color = getImgAvg (img)
    print('avg_color : ', avg_color)
    season = rgb2lab ( avg_color )
    print('lab : ', season)
    
    print(getPersonalColor(season, season))
    
    return '롱스톤'

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000, debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
<ipython-input-31-c02af543b885>:7: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  encoded_img = np.fromstring(binary_decode, dtype = np.uint8)


res :  <Response [200]>
shape :  354 ,  472
[(104, 174), (104, 194), (106, 213), (110, 232), (116, 251), (126, 267), (139, 282), (154, 293), (172, 297), (190, 295), (205, 285), (218, 271), (229, 256), (236, 238), (241, 220), (245, 201), (247, 182), (116, 163), (127, 158), (140, 157), (153, 161), (164, 167), (193, 169), (206, 164), (218, 162), (230, 163), (239, 170)]


127.0.0.1 - - [03/Aug/2022 15:24:17] "POST /test/personalcolor HTTP/1.1" 200 -


avg_color :  [222.57257476294686, 195.62484804279126, 192.20423048869452]
lab :  [81.1517, 8.9478, 5.1987]
겨울쿨톤


In [ ]:
# imageUrl_name = 'test_img_black_1'
imageUrl_name = 'test_img_black_2'
imageUrl_name = 'test_img_brown_1'

# imageUrl = 'img/test_img_black_1.jpg'
# imageUrl = 'img/test_img_black_2.jpg'
imageUrl = 'img/' + imageUrl_name + '.png'

# 이미지 불러오기
img = cv2.imread(imageUrl)
img_api = open(imageUrl, 'rb')
binary_decode = base64.b64decode(img) # image binary 

res = getfacialPointsJsonInKakaoAPI(img_api)

In [3]:
#jpg

def getFaceImg(img, res):
    result_img = img.copy()

    h_o, w_o, c = result_img.shape

    print('shape : ', w_o, ", ", h_o)

    json_faces = json.loads(str(res.json()['result']['faces'])[1:-1].replace('\'', '\"'))

    x = json_faces['x']
    y = json_faces['y']
    w = json_faces['w']
    h = json_faces['h']

    json_facial_points = json_faces['facial_points']

    erase_object(result_img, json_facial_points['left_eyebrow'], w_o, h_o)
    erase_object(result_img, json_facial_points['right_eyebrow'], w_o, h_o)
    erase_object(result_img, json_facial_points['left_eye'], w_o, h_o)
    erase_object(result_img, json_facial_points['right_eye'], w_o, h_o)
    erase_object(result_img, json_facial_points['lip'], w_o, h_o)
    # erase_object(result_img, , w_o, h_o)

    polygon_face = []
    # polygon_face.append(json_facial_points['right_eyebrow'])
    # polygon_face.append(json_facial_points['left_eyebrow'])


    for position in json_facial_points['jaw']:
        point = (int(position[0]*w_o), int(position[1]*h_o))
        polygon_face.append(point)

    for position in json_facial_points['right_eyebrow']:
        point = (int(position[0]*w_o), int(position[1]*h_o))
        polygon_face.append(point)

    for position in json_facial_points['left_eyebrow']:
        point = (int(position[0]*w_o), int(position[1]*h_o))
        polygon_face.append(point)


    result_img = np.asarray(result_img)

    # convert to numpy (for convenience)
    result_img = np.asarray(result_img)

    # create mask
    polygon = [(444,203),(623,243),(691,177),(581,26),(482,42)]

    # create new image ("1-bit pixels, black and white", (width, height), "default color")
    mask_img = Image.new('1', (result_img.shape[1], result_img.shape[0]), 0)

    ImageDraw.Draw(mask_img).polygon(polygon_face, outline=1, fill=1)
    mask = np.array(mask_img)

    # assemble new image (uint8: 0-255)
    new_img_array = np.empty(result_img.shape, dtype='uint8')

    # copy color values (RGB)
    new_img_array[:,:,:3] = result_img[:,:,:3]

    # filtering image by mask
    new_img_array[:,:,0] = new_img_array[:,:,0] * mask
    new_img_array[:,:,1] = new_img_array[:,:,1] * mask
    new_img_array[:,:,2] = new_img_array[:,:,2] * mask

    # back to Image from numpy
    result_img = Image.fromarray(new_img_array, "RGB")

    resultimg_url = 'img/' + 'deviceTest' + '_result.png'
    cv2.imwrite(resultimg_url, np.float32(result_img))

    return np.float32(result_img)

In [2]:
def erase_object(result_img, list, w_o, h_o): # 이미지, 원하는 좌표가 들어간 배열, 이미지 크기
    x_left = w_o
    x_right = 0
    y_up = h_o
    y_down = 0

    for x_temp, y_temp in list:
        y_temp = (int)(y_temp*h_o)
        x_temp = (int)(x_temp*w_o)
        if x_left > x_temp:
            x_left = x_temp
        if x_right < x_temp:
            x_right = x_temp
        if y_up > y_temp:
            y_up = y_temp
        if y_down < y_temp:
            y_down = y_temp

    cv2.rectangle(result_img, (x_left, y_up), (x_right, y_down), (0, 0, 0), -1)

In [4]:
##################################################################
# kakao API로 이미지 얼굴 특징 좌표 값 받아오기 -> 파라미터는 바이너리 파일
# return : facial_points_json / <class 'str'>

# jaw 검출된 턱선의 17개의 좌표 배열, 0 ~ 1.0 사이의 (x,y) 값의 배열
# right_eyebrow 검출된 오른쪽 눈썹의 5개의 좌표 배열, 0 ~ 1.0 사이의 (x,y) 값의 배열
# left_eyebrow 검출된 왼쪽 눈썹의 5개의 좌표 배열, 0 ~ 1.0 사이의 (x,y) 값의 배열
# nose 검출된 코의 9개의 좌표 배열, 0 ~ 1.0 사이의 (x,y) 값의 배열
# right_eye 검출된 오른쪽 눈의 6개의 좌표 배열, 0 ~ 1.0 사이의 (x,y) 값의 배열
# left_eye 검출된 왼쪽 눈의 6개의 좌표 배열, 0 ~ 1.0 사이의 (x,y) 값의 배열
# lip 검출된 입술의 20개의 좌표 배열, 0 ~ 1.0 사이의 (x,y) 값의 배열
##################################################################


def getfacialPointsJsonInKakaoAPI(image):
    url_kakaoAPI = "dapi.kakao.com/v2/vision/face/detect"

    # headers = {'Content-Type': 'application/json; chearset=utf-8'}

    appkey = "cd01a673937be048c7a932050faed168"

    headers = {
        # requests won't add a boundary if this header is set when you pass files=
        # 'Content-Type': 'multipart/form-data',
        'Authorization': 'KakaoAK cd01a673937be048c7a932050faed168',
    }

    files = {
        'image': image,
    }

    res = requests.post('https://dapi.kakao.com/v2/vision/face/detect', headers=headers, files=files)
    
    return res

In [27]:
##################################################################
# getImgAvg --> 이미지 cv2로 읽는 건 되는데 api로 이미지 받은 것도 되는지는 모르겠네
# --> image로 받는 거 바이너리 파일이니까 진짜 이미지로 변경해주어야함
# return : [R, G, B] / <class 'list'>
##################################################################


# 진또배기

# image = cv2.imread(img_url)

def getImgAvg ( image ) :

    
    # imge_binary To image(npArray)
    
    # 채널을 BGR -> RGB로 변경
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image.reshape((image.shape[0] * image.shape[1], 3)) # height, width 통합

    count_color = 10
    clt = KMeans(n_clusters = count_color)
    clt.fit(image)

    colorList = []
    numLabels = np.arange(0, len(np.unique(clt.labels_)) + 1)
    (hist, _) = np.histogram(clt.labels_, bins=numLabels)

    # normalize the histogram, such that it sums to one
    hist = hist.astype("float")
    hist /= hist.sum()
    
    colorRatioList = list(hist)
    for (colorRatio, color) in zip(colorRatioList, clt.cluster_centers_):
        colorList.append(color.astype(np.int64))
      # print(color, colorRatio)


    avg_color = [0, 0, 0]
    colorRatio_std = -1
    for (color, colorRatio) in zip(colorList, colorRatioList):
        if(color[0] != 0 and color[1] != 0 and color[2] != 0): #제거된 배경은 제외
#             print(color[0], ",  ", color[1], ",  ", color[2], " -> ", colorRatio)
            avg_color[0] += color[0] * colorRatio * 100
            avg_color[1] += color[1] * colorRatio * 100
            avg_color[2] += color[2] * colorRatio * 100
        else:
            colorRatio_std = colorRatio * 100

    avg_color[0] = avg_color[0]/(100-colorRatio_std)
    avg_color[1] = avg_color[1]/(100-colorRatio_std)
    avg_color[2] = avg_color[2]/(100-colorRatio_std)
    
    return avg_color

In [8]:
##################################################################
# cut img Background 
# return : result / <class 'np.array'(img)>
##################################################################


def cutBackground_darkHair(img):
    # 변환 graky
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 임계값 조절
    mask = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)[1]

    # mask
    mask = 255 - mask

    # morphology 적용
    # borderconstant 사용
    kernel = np.ones((3,3), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

    # anti-alias the mask
    # blur alpha channel
    mask = cv2.GaussianBlur(mask, (0,0), sigmaX=2, sigmaY=2, borderType = cv2.BORDER_DEFAULT)

    # linear stretch so that 127.5 goes to 0, but 255 stays 255
    mask = (2*(mask.astype(np.float32))-255.0).clip(0,255).astype(np.uint8)

    # put mask into alpha channel
    result = img.copy()
    result = cv2.cvtColor(result, cv2.COLOR_BGR2BGRA)
    result[:, :, 3] = mask

    return result

In [12]:
##################################################################
# RGB to Lab
# return : [L, A, B] / <class 'list'>
##################################################################


def rgb2lab ( inputColor ) :

   num = 0
   RGB = [0, 0, 0]

   for value in inputColor :
       value = float(value) / 255

       if value > 0.04045 :
           value = ( ( value + 0.055 ) / 1.055 ) ** 2.4
       else :
           value = value / 12.92

       RGB[num] = value * 100
       num = num + 1

   XYZ = [0, 0, 0,]

   X = RGB [0] * 0.4124 + RGB [1] * 0.3576 + RGB [2] * 0.1805
   Y = RGB [0] * 0.2126 + RGB [1] * 0.7152 + RGB [2] * 0.0722
   Z = RGB [0] * 0.0193 + RGB [1] * 0.1192 + RGB [2] * 0.9505
   XYZ[ 0 ] = round( X, 4 )
   XYZ[ 1 ] = round( Y, 4 )
   XYZ[ 2 ] = round( Z, 4 )

   XYZ[ 0 ] = float( XYZ[ 0 ] ) / 95.047         # ref_X =  95.047   Observer= 2°, Illuminant= D65
   XYZ[ 1 ] = float( XYZ[ 1 ] ) / 100.0          # ref_Y = 100.000
   XYZ[ 2 ] = float( XYZ[ 2 ] ) / 108.883        # ref_Z = 108.883

   num = 0
   for value in XYZ :

       if value > 0.008856 :
           value = value ** ( 0.3333333333333333 )
       else :
           value = ( 7.787 * value ) + ( 16 / 116 )

       XYZ[num] = value
       num = num + 1

   Lab = [0, 0, 0]

   L = ( 116 * XYZ[ 1 ] ) - 16
   a = 500 * ( XYZ[ 0 ] - XYZ[ 1 ] )
   b = 200 * ( XYZ[ 1 ] - XYZ[ 2 ] )

   Lab [ 0 ] = round( L, 4 )
   Lab [ 1 ] = round( a, 4 )
   Lab [ 2 ] = round( b, 4 )

   return Lab

In [13]:
##################################################################
# getPersonalColor
# return : "가을웜톤" / <class 'String'>
##################################################################

def getPersonalColor(personal, season):
    
    result = ""
    
    if(personal[1] > personal[2]):
        result += "쿨톤"
        
        if(season[0] > 50):
            result = "겨울" + result
        else:
            result = "여름" + result
            
    else:
        result += "웜톤"
        
        if(season[0] > 50):
            result = "봄" + result
        else:
            result = "가을" + result
            
    return result